In [46]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Dictionnaire de correspondance entre les noms des partis politiques et leurs identifiants numériques
id_to_party_dict = {
    1: "Lutte Ouvrière",
    2: "Parti Communiste Français",
    3: "La République En Marche",
    4: "Résistons",
    5: "Rassemblement National",
    6: "Reconquête",
    7: "La France Insoumise",
    8: "Parti Socialiste",
    9: "Europe Écologie Les Verts",
    10: "Les Républicains",
    11: "Nouveau Parti Anticapitaliste",
    12: "Debout La France"
}

# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432", 
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Requête SQL pour sélectionner les données de la table election_2022_t1
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête SQL pour sélectionner les données de la table économique
query_economy = """
    SELECT *
    FROM economie
"""
df_economy = pd.read_sql(query_economy, conn)

# Fermeture de la connexion
conn.close()

# Mapper les noms des partis politiques aux identifiants numériques
party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}
df_election["winner"] = df_election["winner"].map(party_to_id_dict)

# Remplacer les valeurs manquantes dans la colonne "winner" par une valeur arbitraire
df_election["winner"].fillna(-1, inplace=True)

# Convertir la colonne "winner" en float
df_election["winner"] = df_election["winner"].astype(float)

# Fusionner les données électorales avec les données économiques
df_combined = pd.merge(df_election, df_economy, on="code_postal")

# Encodage des variables catégorielles
label_encoder = LabelEncoder()

# Séparation des features et de la target
X = df_combined.drop(columns=["Libellé de la commune", "winner"])
y = df_combined["winner"]

# Division des données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle de régression logistique multinomiale
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation de la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur l'ensemble de test:", accuracy)

# Utilisation du modèle pour prédire le parti gagnant dans la colonne "winner" pour le département de l'Yonne
# Vous devez remplacer les valeurs de X_yonne avec les données spécifiques de l'Yonne
# Filtrer les données de test pour ne conserver que celles du département 89
X_test_yonne = X_test[X_test["code_postal"] == 89]
y_test_yonne = y_test[X_test["code_postal"] == 89]

# Prédiction sur les données de test de l'Yonne
y_pred_yonne = model.predict(X_test_yonne)

# Évaluation de la précision du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_test_yonne, y_pred_yonne)
print("Précision du modèle sur les données de l'Yonne:", accuracy_yonne)

# Afficher les prédictions du modèle sur les données de l'Yonne
print("Prédictions du modèle sur les données de l'Yonne:")
print(y_pred_yonne)


/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_8412/3258273224.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_8412/3258273224.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economy = pd.read_sql(query_economy, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_8412/3258273224.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 

Précision du modèle sur l'ensemble de test: 0.5742331288343558
Précision du modèle sur les données de l'Yonne: 0.835820895522388
Prédictions du modèle sur les données de l'Yonne:
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]


/Users/erwanbuisson/Documents/GitHub/mspr/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
